In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [15]:
df_features = pd.read_csv('train_features.csv',index_col='sig_id')
df_test_features = pd.read_csv('test_features.csv',index_col='sig_id')
df_sample_submission = pd.read_csv('sample_submission.csv', index_col='sig_id')
df_targets = pd.read_csv('train_targets_scored.csv',index_col='sig_id')
df_targets_nonscored = pd.read_csv('train_targets_nonscored.csv',index_col='sig_id')
df = pd.concat([df_features, df_targets], axis=1)
df_features

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_fffb1ceed,trt_cp,24,D2,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,0.5773,...,0.1969,0.0262,-0.8121,0.3434,0.5372,-0.3246,0.0631,0.9171,0.5258,0.4680
id_fffb70c0c,trt_cp,24,D2,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,-0.2252,...,0.4286,0.4426,0.0423,-0.3195,-0.8086,-0.9798,-0.2084,-0.1224,-0.2715,0.3689
id_fffc1c3f4,ctl_vehicle,48,D2,0.3942,0.3756,0.3109,-0.7389,0.5505,-0.0159,-0.2541,...,0.5409,0.3755,0.7343,0.2807,0.4116,0.6422,0.2256,0.7592,0.6656,0.3808


In [16]:
def change_df(df):
    df['cp_type'] = df['cp_type'].apply(lambda x: 1 if x == "ctl_vehicle" else 0)
    df['cp_dose'] = df['cp_dose'].apply(lambda x: 1 if x == "D2" else 0)
    df['cp_time'] = df['cp_time'].apply(lambda x: (x - 24)/48)
    return df
df_features = change_df(df_features)
df_test_features = change_df(df_test_features)

In [17]:
df_targets = df_targets[df_features['cp_type'] == 0]
df_targets_nonscored = df_targets_nonscored[df_features['cp_type'] == 0]
df_features = df_features[df_features['cp_type'] == 0]

In [18]:
!pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.1-py3-none-any.whl pytorch-tabnet
!pip install ../input/autograd

Looking in links: /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.1-py3-none-any.whl


ERROR: Could not install packages due to an EnvironmentError: Invalid URL '/kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.1-py3-none-any.whl': No schema supplied. Perhaps you meant http:///kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.1-py3-none-any.whl?

ERROR: Invalid requirement: '../input/autograd'


In [19]:
pip install "iterative-stratification"

Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install pytorch-tabnet

  Using cached pytorch_tabnet-2.0.1-py3-none-any.whl (30 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torch<2.0,>=1.2 (from pytorch-tabnet) (from versions: 0.1.2, 0.1.2.post1, 0.1.2.post2)
ERROR: No matching distribution found for torch<2.0,>=1.2 (from pytorch-tabnet)


In [21]:
import sys

sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from pytorch_tabnet.tab_model import TabNetRegressor
from tqdm import *
from tqdm.keras import TqdmCallback
from imblearn.over_sampling import *

ModuleNotFoundError: No module named 'pytorch_tabnet'

In [22]:
from sklearn.feature_selection import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.decomposition import *
from sklearn.ensemble import *
from sklearn.metrics import *

In [23]:
X = df_features.values
y = df_targets.values
y_nonscored = df_targets_nonscored.values
X_test = df_test_features.values

In [24]:
pca_g = PCA(n_components=50)

X_pca_g = pca_g.fit_transform(X[:, 3:-100])
X_test_pca_g = pca_g.transform(X_test[:, 3:-100])

pca_c = PCA(n_components=15)

X_pca_c = pca_c.fit_transform(X[:, -100:])
X_test_pca_c = pca_c.transform(X_test[:, -100:])

In [25]:
from scipy import stats
X_stats_1 = stats.describe(X[:, 3:-100], axis=1)
X_stats_2 = stats.describe(X[:, -100:], axis=1)
X_stats_3 = stats.describe(X[:, 3:], axis=1)
X_stats = np.vstack((X_stats_1.mean, X_stats_1.variance, X_stats_1.skewness, X_stats_1.kurtosis, X_stats_2.mean, X_stats_2.variance, X_stats_2.skewness, X_stats_2.kurtosis, X_stats_3.mean, X_stats_3.variance, X_stats_3.skewness, X_stats_3.kurtosis)).T
X_stats_1 = stats.describe(X_test[:, 3:-100], axis=1)
X_stats_2 = stats.describe(X_test[:, -100:], axis=1)
X_stats_3 = stats.describe(X_test[:, 3:], axis=1)
X_test_stats = np.vstack((X_stats_1.mean, X_stats_1.variance, X_stats_1.skewness, X_stats_1.kurtosis, X_stats_2.mean, X_stats_2.variance, X_stats_2.skewness, X_stats_2.kurtosis, X_stats_3.mean, X_stats_3.variance, X_stats_3.skewness, X_stats_3.kurtosis)).T

In [26]:
selector = VarianceThreshold(threshold=0.5)
X = np.concatenate([X[:, 1:3], selector.fit_transform(X[:, 3:])], axis=1)
X_test = np.concatenate([X_test[:, 1:3], selector.transform(X_test[:, 3:])], axis=1)

#X = X[:, top_feats]
#X_test = X_test[:, top_feats]

In [27]:
for i in tqdm(range(X.shape[1]-2)):
    scaler = QuantileTransformer(output_distribution="normal")
    X[:, i+2] = scaler.fit_transform(X[:, i+2].reshape(-1, 1)).reshape(-1)
    X_test[:, i+2] = scaler.transform(X_test[:, i+2].reshape(-1, 1)).reshape(-1)

NameError: name 'tqdm' is not defined

In [ ]:
X = np.concatenate([X, X_stats, X_pca_g, X_pca_c], axis=1)
X_test = np.concatenate([X_test, X_test_stats, X_test_pca_g, X_test_pca_c], axis=1)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float).to(device)
        self.targets = torch.tensor(targets, dtype=torch.float).to(device)
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

class MoATestDataset:
    def __init__(self, features):
        self.features = torch.tensor(features, dtype=torch.float).to(device)
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        return self.features[idx]

In [ ]:
# Lookahead implementation from https://github.com/rwightman/pytorch-image-models/blob/master/timm/optim/lookahead.py

""" Lookahead Optimizer Wrapper.
Implementation modified from: https://github.com/alphadl/lookahead.pytorch
Paper: `Lookahead Optimizer: k steps forward, 1 step back` - https://arxiv.org/abs/1907.08610
"""
import torch
from torch.optim.optimizer import Optimizer
from torch.optim import Adam
from collections import defaultdict

class Lookahead(Optimizer):
    def __init__(self, base_optimizer, alpha=0.5, k=6):
        if not 0.0 <= alpha <= 1.0:
            raise ValueError(f'Invalid slow update rate: {alpha}')
        if not 1 <= k:
            raise ValueError(f'Invalid lookahead steps: {k}')
        defaults = dict(lookahead_alpha=alpha, lookahead_k=k, lookahead_step=0)
        self.base_optimizer = base_optimizer
        self.param_groups = self.base_optimizer.param_groups
        self.defaults = base_optimizer.defaults
        self.defaults.update(defaults)
        self.state = defaultdict(dict)
        # manually add our defaults to the param groups
        for name, default in defaults.items():
            for group in self.param_groups:
                group.setdefault(name, default)

    def update_slow(self, group):
        for fast_p in group["params"]:
            if fast_p.grad is None:
                continue
            param_state = self.state[fast_p]
            if 'slow_buffer' not in param_state:
                param_state['slow_buffer'] = torch.empty_like(fast_p.data)
                param_state['slow_buffer'].copy_(fast_p.data)
            slow = param_state['slow_buffer']
            slow.add_(group['lookahead_alpha'], fast_p.data - slow)
            fast_p.data.copy_(slow)

    def sync_lookahead(self):
        for group in self.param_groups:
            self.update_slow(group)

    def step(self, closure=None):
        # print(self.k)
        #assert id(self.param_groups) == id(self.base_optimizer.param_groups)
        loss = self.base_optimizer.step(closure)
        for group in self.param_groups:
            group['lookahead_step'] += 1
            if group['lookahead_step'] % group['lookahead_k'] == 0:
                self.update_slow(group)
        return loss

    def state_dict(self):
        fast_state_dict = self.base_optimizer.state_dict()
        slow_state = {
            (id(k) if isinstance(k, torch.Tensor) else k): v
            for k, v in self.state.items()
        }
        fast_state = fast_state_dict['state']
        param_groups = fast_state_dict['param_groups']
        return {
            'state': fast_state,
            'slow_state': slow_state,
            'param_groups': param_groups,
        }

    def load_state_dict(self, state_dict):
        fast_state_dict = {
            'state': state_dict['state'],
            'param_groups': state_dict['param_groups'],
        }
        self.base_optimizer.load_state_dict(fast_state_dict)

        # We want to restore the slow state, but share param_groups reference
        # with base_optimizer. This is a bit redundant but least code
        slow_state_new = False
        if 'slow_state' not in state_dict:
            print('Loading state_dict from optimizer without Lookahead applied.')
            state_dict['slow_state'] = defaultdict(dict)
            slow_state_new = True
        slow_state_dict = {
            'state': state_dict['slow_state'],
            'param_groups': state_dict['param_groups'],  # this is pointless but saves code
        }
        super(Lookahead, self).load_state_dict(slow_state_dict)
        self.param_groups = self.base_optimizer.param_groups  # make both ref same container
        if slow_state_new:
            # reapply defaults to catch missing lookahead specific ones
            for name, default in self.defaults.items():
                for group in self.param_groups:
                    group.setdefault(name, default)

def LookaheadAdam(params, alpha=0.5, k=6, *args, **kwargs):
     adam = Adam(params, *args, **kwargs)
     return Lookahead(adam, alpha, k)

In [ ]:
def train_model(model, optimizer, scheduler, criterion, dataloader):
    model.train()
    final_loss = 0
    
    for i, data in enumerate(dataloader):
        #X_batch, y_batch = [t.to(device) for t in data]
        X_batch, y_batch = data
        optimizer.zero_grad()
        
        inputs, targets = X_batch, y_batch
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        
        final_loss += loss.item()
        scheduler.step()
    final_loss /= len(dataloader)
    
    return final_loss


def validate_model(model, criterion, dataloader):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for i, data in enumerate(dataloader):
        #X_batch, y_batch = [t.to(device) for t in data]        
        X_batch, y_batch = data
        
        inputs, targets = X_batch, y_batch
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def test_model(model, dataloader):
    model.eval()
    preds = []
    
    for data in tqdm(dataloader):
        inputs = data

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds

In [ ]:
import contextlib
from torch.autograd import Function
from collections import OrderedDict
from torch.jit import script
from warnings import warn
import os
import glob
import hashlib
import gc
import time
import requests

def to_one_hot(y, depth=None):
    r"""
    Takes integer with n dims and converts it to 1-hot representation with n + 1 dims.
    The n+1'st dimension will have zeros everywhere but at y'th index, where it will be equal to 1.
    Args:
        y: input integer (IntTensor, LongTensor or Variable) of any shape
        depth (int):  the size of the one hot dimension
    """
    y_flat = y.to(torch.int64).view(-1, 1)
    depth = depth if depth is not None else int(torch.max(y_flat)) + 1
    y_one_hot = torch.zeros(y_flat.size()[0], depth, device=y.device).scatter_(1, y_flat, 1)
    y_one_hot = y_one_hot.view(*(tuple(y.shape) + (-1,)))
    return y_one_hot


def _make_ix_like(input, dim=0):
    d = input.size(dim)
    rho = torch.arange(1, d + 1, device=input.device, dtype=input.dtype)
    view = [1] * input.dim()
    view[0] = -1
    return rho.view(view).transpose(0, dim)


class SparsemaxFunction(Function):
    """
    An implementation of sparsemax (Martins & Astudillo, 2016). See
    :cite:`DBLP:journals/corr/MartinsA16` for detailed description.
    By Ben Peters and Vlad Niculae
    """

    @staticmethod
    def forward(ctx, input, dim=-1):
        """sparsemax: normalizing sparse transform (a la softmax)
        Parameters:
            input (Tensor): any shape
            dim: dimension along which to apply sparsemax
        Returns:
            output (Tensor): same shape as input
        """
        ctx.dim = dim
        max_val, _ = input.max(dim=dim, keepdim=True)
        input -= max_val  # same numerical stability trick as for softmax
        tau, supp_size = SparsemaxFunction._threshold_and_support(input, dim=dim)
        output = torch.clamp(input - tau, min=0)
        ctx.save_for_backward(supp_size, output)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        supp_size, output = ctx.saved_tensors
        dim = ctx.dim
        grad_input = grad_output.clone()
        grad_input[output == 0] = 0

        v_hat = grad_input.sum(dim=dim) / supp_size.to(output.dtype).squeeze()
        v_hat = v_hat.unsqueeze(dim)
        grad_input = torch.where(output != 0, grad_input - v_hat, grad_input)
        return grad_input, None


    @staticmethod
    def _threshold_and_support(input, dim=-1):
        """Sparsemax building block: compute the threshold
        Args:
            input: any dimension
            dim: dimension along which to apply the sparsemax
        Returns:
            the threshold value
        """

        input_srt, _ = torch.sort(input, descending=True, dim=dim)
        input_cumsum = input_srt.cumsum(dim) - 1
        rhos = _make_ix_like(input, dim)
        support = rhos * input_srt > input_cumsum

        support_size = support.sum(dim=dim).unsqueeze(dim)
        tau = input_cumsum.gather(dim, support_size - 1)
        tau /= support_size.to(input.dtype)
        return tau, support_size


sparsemax = lambda input, dim=-1: SparsemaxFunction.apply(input, dim)
sparsemoid = lambda input: (0.5 * input + 0.5).clamp_(0, 1)


class Entmax15Function(Function):
    """
    An implementation of exact Entmax with alpha=1.5 (B. Peters, V. Niculae, A. Martins). See
    :cite:`https://arxiv.org/abs/1905.05702 for detailed description.
    Source: https://github.com/deep-spin/entmax
    """

    @staticmethod
    def forward(ctx, input, dim=-1):
        ctx.dim = dim

        max_val, _ = input.max(dim=dim, keepdim=True)
        input = input - max_val  # same numerical stability trick as for softmax
        input = input / 2  # divide by 2 to solve actual Entmax

        tau_star, _ = Entmax15Function._threshold_and_support(input, dim)
        output = torch.clamp(input - tau_star, min=0) ** 2
        ctx.save_for_backward(output)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        Y, = ctx.saved_tensors
        gppr = Y.sqrt()  # = 1 / g'' (Y)
        dX = grad_output * gppr
        q = dX.sum(ctx.dim) / gppr.sum(ctx.dim)
        q = q.unsqueeze(ctx.dim)
        dX -= q * gppr
        return dX, None

    @staticmethod
    def _threshold_and_support(input, dim=-1):
        Xsrt, _ = torch.sort(input, descending=True, dim=dim)

        rho = _make_ix_like(input, dim)
        mean = Xsrt.cumsum(dim) / rho
        mean_sq = (Xsrt ** 2).cumsum(dim) / rho
        ss = rho * (mean_sq - mean ** 2)
        delta = (1 - ss) / rho

        # NOTE this is not exactly the same as in reference algo
        # Fortunately it seems the clamped values never wrongly
        # get selected by tau <= sorted_z. Prove this!
        delta_nz = torch.clamp(delta, 0)
        tau = mean - torch.sqrt(delta_nz)

        support_size = (tau <= Xsrt).sum(dim).unsqueeze(dim)
        tau_star = tau.gather(dim, support_size - 1)
        return tau_star, support_size


class Entmoid15(Function):
    """ A highly optimized equivalent of labda x: Entmax15([x, 0]) """

    @staticmethod
    def forward(ctx, input):
        output = Entmoid15._forward(input)
        ctx.save_for_backward(output)
        return output

    @staticmethod
    @script
    def _forward(input):
        input, is_pos = abs(input), input >= 0
        tau = (input + torch.sqrt(F.relu(8 - input ** 2))) / 2
        tau.masked_fill_(tau <= input, 2.0)
        y_neg = 0.25 * F.relu(tau - input, inplace=True) ** 2
        return torch.where(is_pos, 1 - y_neg, y_neg)

    @staticmethod
    def backward(ctx, grad_output):
        return Entmoid15._backward(ctx.saved_tensors[0], grad_output)

    @staticmethod
    @script
    def _backward(output, grad_output):
        gppr0, gppr1 = output.sqrt(), (1 - output).sqrt()
        grad_input = grad_output * gppr0
        q = grad_input / (gppr0 + gppr1)
        grad_input -= q * gppr0
        return grad_input


entmax15 = lambda input, dim=-1: Entmax15Function.apply(input, dim)
entmoid15 = Entmoid15.apply


class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, *args, **kwargs):
        return self.func(*args, **kwargs)


class ModuleWithInit(nn.Module):
    """ Base class for pytorch module with data-aware initializer on first batch """
    def __init__(self):
        super().__init__()
        self._is_initialized_tensor = nn.Parameter(torch.tensor(0, dtype=torch.uint8), requires_grad=False)
        self._is_initialized_bool = None
        # Note: this module uses a separate flag self._is_initialized so as to achieve both
        # * persistence: is_initialized is saved alongside model in state_dict
        # * speed: model doesn't need to cache
        # please DO NOT use these flags in child modules

    def initialize(self, *args, **kwargs):
        """ initialize module tensors using first batch of data """
        raise NotImplementedError("Please implement ")

    def __call__(self, *args, **kwargs):
        if self._is_initialized_bool is None:
            self._is_initialized_bool = bool(self._is_initialized_tensor.item())
        if not self._is_initialized_bool:
            self.initialize(*args, **kwargs)
            self._is_initialized_tensor.data[...] = 1
            self._is_initialized_bool = True
        return super().__call__(*args, **kwargs)

def download(url, filename, delete_if_interrupted=True, chunk_size=4096):
    """ saves file from url to filename with a fancy progressbar """
    try:
        with open(filename, "wb") as f:
            print("Downloading {} > {}".format(url, filename))
            response = requests.get(url, stream=True)
            total_length = response.headers.get('content-length')

            if total_length is None:  # no content length header
                f.write(response.content)
            else:
                total_length = int(total_length)
                with tqdm(total=total_length) as progressbar:
                    for data in response.iter_content(chunk_size=chunk_size):
                        if data:  # filter-out keep-alive chunks
                            f.write(data)
                            progressbar.update(len(data))
    except Exception as e:
        if delete_if_interrupted:
            print("Removing incomplete download {}.".format(filename))
            os.remove(filename)
        raise e
    return filename


def iterate_minibatches(*tensors, batch_size, shuffle=True, epochs=1,
                        allow_incomplete=True, callback=lambda x:x):
    indices = np.arange(len(tensors[0]))
    upper_bound = int((np.ceil if allow_incomplete else np.floor) (len(indices) / batch_size)) * batch_size
    epoch = 0
    while True:
        if shuffle:
            np.random.shuffle(indices)
        for batch_start in callback(range(0, upper_bound, batch_size)):
            batch_ix = indices[batch_start: batch_start + batch_size]
            batch = [tensor[batch_ix] for tensor in tensors]
            yield batch if len(tensors) > 1 else batch[0]
        epoch += 1
        if epoch >= epochs:
            break


def process_in_chunks(function, *args, batch_size, out=None, **kwargs):
    """
    Computes output by applying batch-parallel function to large data tensor in chunks
    :param function: a function(*[x[indices, ...] for x in args]) -> out[indices, ...]
    :param args: one or many tensors, each [num_instances, ...]
    :param batch_size: maximum chunk size processed in one go
    :param out: memory buffer for out, defaults to torch.zeros of appropriate size and type
    :returns: function(data), computed in a memory-efficient way
    """
    total_size = args[0].shape[0]
    first_output = function(*[x[0: batch_size] for x in args])
    output_shape = (total_size,) + tuple(first_output.shape[1:])
    if out is None:
        out = torch.zeros(*output_shape, dtype=first_output.dtype, device=first_output.device,
                          layout=first_output.layout, **kwargs)

    out[0: batch_size] = first_output
    for i in range(batch_size, total_size, batch_size):
        batch_ix = slice(i, min(i + batch_size, total_size))
        out[batch_ix] = function(*[x[batch_ix] for x in args])
    return out


def check_numpy(x):
    """ Makes sure x is a numpy array """
    if isinstance(x, torch.Tensor):
        x = x.detach().cpu().numpy()
    x = np.asarray(x)
    assert isinstance(x, np.ndarray)
    return x


@contextlib.contextmanager
def nop_ctx():
    yield None


def get_latest_file(pattern):
    list_of_files = glob.glob(pattern) # * means all if need specific format then *.csv
    assert len(list_of_files) > 0, "No files found: " + pattern
    return max(list_of_files, key=os.path.getctime)


def md5sum(fname):
    """ Computes mdp checksum of a file """
    hash_md5 = hashlib.md5()
    with open(fname, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()


def free_memory(sleep_time=0.1):
    """ Black magic function to free torch memory and some jupyter whims """
    gc.collect()
    torch.cuda.synchronize()
    gc.collect()
    torch.cuda.empty_cache()
    time.sleep(sleep_time)

def to_float_str(element):
    try:
        return str(float(element))
    except ValueError:
        return element
    
class ODST(ModuleWithInit):
    def __init__(self, in_features, num_trees, depth=6, tree_dim=1, flatten_output=True,
                 choice_function=sparsemax, bin_function=sparsemoid,
                 initialize_response_=nn.init.normal_, initialize_selection_logits_=nn.init.uniform_,
                 threshold_init_beta=1.0, threshold_init_cutoff=1.0,
                 ):
        """
        Oblivious Differentiable Sparsemax Trees. http://tinyurl.com/odst-readmore
        One can drop (sic!) this module anywhere instead of nn.Linear
        :param in_features: number of features in the input tensor
        :param num_trees: number of trees in this layer
        :param tree_dim: number of response channels in the response of individual tree
        :param depth: number of splits in every tree
        :param flatten_output: if False, returns [..., num_trees, tree_dim],
            by default returns [..., num_trees * tree_dim]
        :param choice_function: f(tensor, dim) -> R_simplex computes feature weights s.t. f(tensor, dim).sum(dim) == 1
        :param bin_function: f(tensor) -> R[0, 1], computes tree leaf weights
        :param initialize_response_: in-place initializer for tree output tensor
        :param initialize_selection_logits_: in-place initializer for logits that select features for the tree
        both thresholds and scales are initialized with data-aware init (or .load_state_dict)
        :param threshold_init_beta: initializes threshold to a q-th quantile of data points
            where q ~ Beta(:threshold_init_beta:, :threshold_init_beta:)
            If this param is set to 1, initial thresholds will have the same distribution as data points
            If greater than 1 (e.g. 10), thresholds will be closer to median data value
            If less than 1 (e.g. 0.1), thresholds will approach min/max data values.
        :param threshold_init_cutoff: threshold log-temperatures initializer, \in (0, inf)
            By default(1.0), log-remperatures are initialized in such a way that all bin selectors
            end up in the linear region of sparse-sigmoid. The temperatures are then scaled by this parameter.
            Setting this value > 1.0 will result in some margin between data points and sparse-sigmoid cutoff value
            Setting this value < 1.0 will cause (1 - value) part of data points to end up in flat sparse-sigmoid region
            For instance, threshold_init_cutoff = 0.9 will set 10% points equal to 0.0 or 1.0
            Setting this value > 1.0 will result in a margin between data points and sparse-sigmoid cutoff value
            All points will be between (0.5 - 0.5 / threshold_init_cutoff) and (0.5 + 0.5 / threshold_init_cutoff)
        """
        super().__init__()
        self.depth, self.num_trees, self.tree_dim, self.flatten_output = depth, num_trees, tree_dim, flatten_output
        self.choice_function, self.bin_function = choice_function, bin_function
        self.threshold_init_beta, self.threshold_init_cutoff = threshold_init_beta, threshold_init_cutoff

        self.response = nn.Parameter(torch.zeros([num_trees, tree_dim, 2 ** depth]), requires_grad=True)
        initialize_response_(self.response)

        self.feature_selection_logits = nn.Parameter(
            torch.zeros([in_features, num_trees, depth]), requires_grad=True
        )
        initialize_selection_logits_(self.feature_selection_logits)

        self.feature_thresholds = nn.Parameter(
            torch.full([num_trees, depth], float('nan'), dtype=torch.float32), requires_grad=True
        )  # nan values will be initialized on first batch (data-aware init)

        self.log_temperatures = nn.Parameter(
            torch.full([num_trees, depth], float('nan'), dtype=torch.float32), requires_grad=True
        )

        # binary codes for mapping between 1-hot vectors and bin indices
        with torch.no_grad():
            indices = torch.arange(2 ** self.depth)
            offsets = 2 ** torch.arange(self.depth)
            bin_codes = (indices.view(1, -1) // offsets.view(-1, 1) % 2).to(torch.float32)
            bin_codes_1hot = torch.stack([bin_codes, 1.0 - bin_codes], dim=-1)
            self.bin_codes_1hot = nn.Parameter(bin_codes_1hot, requires_grad=False)
            # ^-- [depth, 2 ** depth, 2]

    def forward(self, input):
        assert len(input.shape) >= 2
        if len(input.shape) > 2:
            return self.forward(input.view(-1, input.shape[-1])).view(*input.shape[:-1], -1)
        # new input shape: [batch_size, in_features]

        feature_logits = self.feature_selection_logits
        feature_selectors = self.choice_function(feature_logits, dim=0)
        # ^--[in_features, num_trees, depth]

        feature_values = torch.einsum('bi,ind->bnd', input, feature_selectors)
        # ^--[batch_size, num_trees, depth]

        threshold_logits = (feature_values - self.feature_thresholds) * torch.exp(-self.log_temperatures)

        threshold_logits = torch.stack([-threshold_logits, threshold_logits], dim=-1)
        # ^--[batch_size, num_trees, depth, 2]

        bins = self.bin_function(threshold_logits)
        # ^--[batch_size, num_trees, depth, 2], approximately binary

        bin_matches = torch.einsum('btds,dcs->btdc', bins, self.bin_codes_1hot)
        # ^--[batch_size, num_trees, depth, 2 ** depth]

        response_weights = torch.prod(bin_matches, dim=-2)
        # ^-- [batch_size, num_trees, 2 ** depth]

        response = torch.einsum('bnd,ncd->bnc', response_weights, self.response)
        # ^-- [batch_size, num_trees, tree_dim]

        return response.flatten(1, 2) if self.flatten_output else response

    def initialize(self, input, eps=1e-6):
        # data-aware initializer
        assert len(input.shape) == 2
        if input.shape[0] < 1000:
            warn("Data-aware initialization is performed on less than 1000 data points. This may cause instability."
                 "To avoid potential problems, run this model on a data batch with at least 1000 data samples."
                 "You can do so manually before training. Use with torch.no_grad() for memory efficiency.")
        with torch.no_grad():
            feature_selectors = self.choice_function(self.feature_selection_logits, dim=0)
            # ^--[in_features, num_trees, depth]

            feature_values = torch.einsum('bi,ind->bnd', input, feature_selectors)
            # ^--[batch_size, num_trees, depth]

            # initialize thresholds: sample random percentiles of data
            percentiles_q = 100 * np.random.beta(self.threshold_init_beta, self.threshold_init_beta,
                                                 size=[self.num_trees, self.depth])
            self.feature_thresholds.data[...] = torch.as_tensor(
                list(map(np.percentile, check_numpy(feature_values.flatten(1, 2).t()), percentiles_q.flatten())),
                dtype=feature_values.dtype, device=feature_values.device
            ).view(self.num_trees, self.depth)

            # init temperatures: make sure enough data points are in the linear region of sparse-sigmoid
            temperatures = np.percentile(check_numpy(abs(feature_values - self.feature_thresholds)),
                                         q=100 * min(1.0, self.threshold_init_cutoff), axis=0)

            # if threshold_init_cutoff > 1, scale everything down by it
            temperatures /= max(1.0, self.threshold_init_cutoff)
            self.log_temperatures.data[...] = torch.log(torch.as_tensor(temperatures) + eps)

    def __repr__(self):
        return "{}(in_features={}, num_trees={}, depth={}, tree_dim={}, flatten_output={})".format(
            self.__class__.__name__, self.feature_selection_logits.shape[0],
            self.num_trees, self.depth, self.tree_dim, self.flatten_output
        )
    
class DenseBlock(nn.Sequential):
    def __init__(self, input_dim, layer_dim, num_layers, tree_dim=1, max_features=None,
                 input_dropout=0.0, flatten_output=True, Module=ODST, **kwargs):
        layers = []
        for i in range(num_layers):
            oddt = Module(input_dim, layer_dim, tree_dim=tree_dim, flatten_output=True, **kwargs)
            input_dim = min(input_dim + layer_dim * tree_dim, max_features or float('inf'))
            layers.append(oddt)

        super().__init__(*layers)
        self.num_layers, self.layer_dim, self.tree_dim = num_layers, layer_dim, tree_dim
        self.max_features, self.flatten_output = max_features, flatten_output
        self.input_dropout = input_dropout

    def forward(self, x):
        initial_features = x.shape[-1]
        for layer in self:
            layer_inp = x
            if self.max_features is not None:
                tail_features = min(self.max_features, layer_inp.shape[-1]) - initial_features
                if tail_features != 0:
                    layer_inp = torch.cat([layer_inp[..., :initial_features], layer_inp[..., -tail_features:]], dim=-1)
            if self.training and self.input_dropout:
                layer_inp = F.dropout(layer_inp, self.input_dropout)
            h = layer(layer_inp)
            x = torch.cat([x, h], dim=-1)

        outputs = x[..., initial_features:]
        if not self.flatten_output:
            outputs = outputs.view(*outputs.shape[:-1], self.num_layers * self.layer_dim, self.tree_dim)
        return outputs

In [ ]:

@torch.jit.script
def mish(input):
    '''
    Applies the mish function element-wise:
    mish(x) = x * tanh(softplus(x)) = x * tanh(ln(1 + exp(x)))
    See additional documentation for mish class.
    '''
    return input * torch.tanh(F.softplus(input))

def swish(x):
    return x * F.sigmoid(x)

top_feats = [  1,   2,   3,   4,   5,   6,   7,   9,  11,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  29,  30,  31,
        32,  33,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  46,
        47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  58,  59,  60,
        61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
        74,  75,  76,  78,  79,  80,  81,  82,  83,  84,  86,  87,  88,
        89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
       102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
       115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
       184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197,
       198, 199, 200, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212,
       213, 214, 215, 216, 217, 218, 219, 220, 221, 223, 224, 225, 226,
       227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239,
       240, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253,
       254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266,
       267, 268, 269, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280,
       281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 294,
       295, 296, 298, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,
       310, 311, 312, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323,
       324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336,
       337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349,
       350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362,
       363, 364, 365, 366, 367, 368, 369, 370, 371, 374, 375, 376, 377,
       378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 390, 391,
       392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404,
       405, 406, 407, 408, 409, 411, 412, 413, 414, 415, 416, 417, 418,
       419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431,
       432, 434, 435, 436, 437, 438, 439, 440, 442, 443, 444, 445, 446,
       447, 448, 449, 450, 453, 454, 456, 457, 458, 459, 460, 461, 462,
       463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475,
       476, 477, 478, 479, 481, 482, 483, 484, 485, 486, 487, 488, 489,
       490, 491, 492, 493, 494, 495, 496, 498, 500, 501, 502, 503, 505,
       506, 507, 509, 510, 511, 512, 513, 514, 515, 518, 519, 520, 521,
       522, 523, 524, 525, 526, 527, 528, 530, 531, 532, 534, 535, 536,
       538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 549, 550, 551,
       552, 554, 557, 559, 560, 561, 562, 565, 566, 567, 568, 569, 570,
       571, 572, 573, 574, 575, 577, 578, 580, 581, 582, 583, 584, 585,
       586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 599,
       600, 601, 602, 606, 607, 608, 609, 611, 612, 613, 615, 616, 617,
       618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630,
       631, 632, 633, 634, 635, 636, 637, 638, 639, 641, 642, 643, 644,
       645, 646, 647, 648, 649, 650, 651, 652, 654, 655, 656, 658, 659,
       660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672,
       673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685,
       686, 687, 688, 689, 691, 692, 693, 694, 695, 696, 697, 699, 700,
       701, 702, 704, 705, 707, 708, 709, 710, 711, 713, 714, 716, 717,
       718, 720, 721, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732,
       733, 734, 735, 737, 738, 739, 740, 742, 743, 744, 745, 746, 747,
       748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 759, 760, 761,
       762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774,
       775, 776, 777, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788,
       789, 790, 792, 793, 794, 795, 796, 797, 798, 800, 801, 802, 803,
       804, 805, 806, 808, 809, 811, 813, 814, 815, 816, 817, 818, 819,
       821, 822, 823, 825, 826, 827, 828, 829, 830, 831, 832, 834, 835,
       837, 838, 839, 840, 841, 842, 845, 846, 847, 848, 850, 851, 852,
       854, 855, 856, 858, 859, 860, 861, 862, 864, 866, 867, 868, 869,
       870, 871, 872, 873, 874]

class MoAModel(nn.Module):
    def __init__(self, num_features, num_targets, num_targets_nonscored, hidden_size):
        super(MoAModel, self).__init__()
        
        
        self.allfeats_1 = nn.Sequential(
            nn.BatchNorm1d(num_features),
            nn.Dropout(0.2),
            nn.utils.weight_norm(nn.Linear(num_features, 1024)),
            nn.PReLU(),
        )
        
        
        self.allfeats_2 = nn.Sequential(
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),
            nn.utils.weight_norm(nn.Linear(1024, 1024)),
            nn.PReLU(),
        )
        
        self.topfeats_1 = nn.Sequential(
            nn.BatchNorm1d(len(top_feats)),
            nn.Dropout(0.1),
            nn.utils.weight_norm(nn.Linear(len(top_feats), 1024)),
            nn.PReLU(),
        )
        
        
        self.combine_1 = nn.Sequential(
            nn.BatchNorm1d(1024 + 2 + 2 + num_features),
            nn.Dropout(0.5),
            nn.utils.weight_norm(nn.Linear(1024 + 2 + 2 + num_features, 1024)),
            nn.PReLU(),
        )
        
        self.classifier = nn.Sequential(
            nn.BatchNorm1d(1024 + 2 + 2),
            nn.Dropout(0.5),
            nn.utils.weight_norm(nn.Linear(1024 + 2 + 2, num_targets)),
        )

        self.time_embedding = nn.Embedding(3, 2)
        self.dose_embedding = nn.Embedding(2, 2)
        
        self.target = 0
    def forward(self, x):
        x_input = x
        
        #x_topfeats = x[:, top_feats]
        
        x_time = self.time_embedding(x[:, 0].long())
        x_dose = self.dose_embedding(x[:, 1].long())
        
        x = self.allfeats_1(x)
        #x = self.allfeats_2(x)
        
        #x_topfeats = self.topfeats_1(x_topfeats)
        #x = (x + x_topfeats)/2
        x = torch.cat([x, x_time, x_dose, x_input], dim=1)
        x = self.combine_1(x)
        
        x = torch.cat([x, x_time, x_dose], dim=1)
        x = self.classifier(x)
        
        return x

class MoAModelHidden(nn.Module):
    def __init__(self, num_features, num_targets, num_targets_nonscored, hidden_size):
        super(MoAModelHidden, self).__init__()
        self.features = nn.Sequential(
            nn.BatchNorm1d(num_features),
            nn.Dropout(0.2),
            nn.utils.weight_norm(nn.Linear(num_features, 1024)),
            nn.PReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),
            nn.utils.weight_norm(nn.Linear(1024, 1024)),
            nn.PReLU(),
        )
        
        self.node = nn.Sequential(nn.BatchNorm1d(num_features), 
                              DenseBlock(num_features, layer_dim = 8, num_layers = 3, tree_dim = 64, depth = 6, input_dropout = 0.3,
                                         flatten_output = True, choice_function = entmax15, bin_function = entmoid15),
                             )
        
        self.classifier = nn.Sequential(
            nn.BatchNorm1d(1024), 
            nn.Dropout(0.5), 
            nn.utils.weight_norm(nn.Linear(1024, num_targets)),
        )
    def forward(self, x):
        #x_input = x
        #x_feature = self.features(x_input)
        #x_node = self.node(x_input)
        #x = torch.cat([x_feature, x_node], dim=1)
        x = self.features(x)
        return self.classifier(x)

In [ ]:
from sklearn.metrics import log_loss
from pytorch_tabnet.metrics import Metric
from sklearn.metrics import roc_auc_score, log_loss

class LogitsLogLoss(Metric):
    """
    LogLoss with sigmoid applied
    """

    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        """
        Compute LogLoss of predictions.

        Parameters
        ----------
        y_true: np.ndarray
            Target matrix or vector
        y_score: np.ndarray
            Score matrix or vector

        Returns
        -------
            float
            LogLoss of predictions vs targets.
        """
        logits = 1 / (1 + np.exp(-y_pred))
        aux = (1-y_true)*np.log(1-logits+1e-15) + y_true*np.log(logits+1e-15)
        return np.mean(-aux)

In [ ]:
def log_loss_numpy(y_true, y_pred):
    loss = 0
    y_pred_clip = np.clip(y_pred, 1e-15, 1 - 1e-15)
    #y_pred_clip = y_pred
    for i in range(y_pred.shape[1]):
        loss += - np.mean(y_true[:, i] * np.log(y_pred_clip[:, i]) + (1 - y_true[:, i]) * np.log(1 - y_pred_clip[:, i]))
    return loss / y_pred.shape[1]
def sigmoid(logits):
    return 1 / (1 + np.exp(-logits))

In [ ]:
import random
KFoldSeed = 1
def test_seed(seed):
    kf = MultilabelStratifiedKFold(5, shuffle=True, random_state=seed)
    for train_index, valid_index in kf.split(X, y):
        y_train = y[train_index]
        y_train = np.sum(y_train, axis=0)
        if y_train[y_train == 0].any():
            return False
    return True

while True:
    KFoldSeed = random.randint(1,1000000000)
    non_empty = test_seed(KFoldSeed)
    print(KFoldSeed, non_empty)
    if non_empty:
        break

In [ ]:
from scipy import optimize
from autograd import grad
import autograd.numpy as anp

def log_loss_numpy_grad(y_true, y_pred):
    loss = 0
    y_pred_clip = anp.clip(y_pred, 1e-15, 1 - 1e-15)
    #y_pred_clip = y_pred
    for i in range(y_pred.shape[1]):
        loss += - anp.mean(y_true[:, i] * anp.log(y_pred_clip[:, i]) + (1 - y_true[:, i]) * anp.log(1 - y_pred_clip[:, i]))
    return loss / y_pred.shape[1]

def ensemble_with_weights(weights, y_preds):
    return np.sum(np.array(y_preds) * np.array(weights).reshape(-1, 1, 1), axis=0)

def ensemble_with_weights_grad(weights, y_preds):
    return anp.sum(y_preds * weights.reshape(-1, 1, 1), axis=0)

def get_best_weights(y_true, y_preds):
    y_preds = anp.array(y_preds, dtype='float64')
    def optimize_func(weights):
        weights, l = weights[:-1], weights[-1]
        return log_loss_numpy_grad(y_true, ensemble_with_weights_grad(weights, y_preds)) - l * (anp.sum(weights) - 1)
    
    
    def optimize_func_nograd(weights):
        return log_loss_numpy(y_true, ensemble_with_weights(weights, y_preds))
    def optimize_func_two(x):
        return optimize_func_nograd([x, 1-x])
    
    grad_L = grad(optimize_func)
    
    def optimize_func_lagrange(params):
        weights, l = params[:-1], params[-1]
        dweights = grad_L(params)
        
        dweights, dl = dweights[:-1], dweights[-1]
        
        return anp.concatenate([dweights, [anp.sum(weights) - 1]])

    


    if len(y_preds) == 2:
        optimized = optimize.minimize_scalar(optimize_func_two, bounds=(0,1), method='bounded')
        print(optimized)
        return [optimized.x, 1-optimized.x]
    
    x0 = [1/len(y_preds)] * (len(y_preds) + 1)
    
    return optimize.fsolve(optimize_func_lagrange, x0)[:-1]
    #x0 = [2/3, 1/3]
    bounds = [(0,1)] * len(y_preds) + [(0, 1)]
    def constr_eq(weights):
        return sum(weights) - 1
    constraints = {'type': 'eq', 'fun': constr_eq}
    
    #optimized = optimize.minimize(optimize_func, x0, method='trust-constr', bounds=bounds, constraints=constraints)    
    optimized = optimize.minimize(optimize_func, x0, bounds=bounds)
    print(optimized)
    return optimized.x[:-1]
    

logloss = nn.BCELoss()
def grad_weights(weights, y_true, y_preds):
    w1 = torch.tensor(weights, requires_grad=True)
    weights = w1.double()
    y_preds = torch.tensor(y_preds).double()
    predsum = torch.sum(y_preds * weights[:-1].view(-1, 1, 1), dim=0)
    
    predsum = torch.clamp(predsum, 0, 1)
    #print(torch.max(stuff), torch.min(stuff))
    loss = (logloss(predsum, torch.tensor(y_true).double())) - weights[-1] * (torch.sum(weights[:-1]) - 1)
    loss.backward()
    return w1.grad.numpy()

def torch_get_best_weights(y_true, y_preds):
    def grad_pred(weights):
        gradw = grad_weights(weights, y_true, y_preds)
        print(weights, gradw)
        return list(gradw[:-1]) + [sum(weights[:-1]) - 1]

    x0 = [1/len(y_preds)] * (len(y_preds) + 1)
    return optimize.fsolve(grad_stuff, x0)

In [28]:
def train_nn(model, trainloader, validloader, epochs, lr):
    
    model.to(device)
    #optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)
    optimizer = LookaheadAdam(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs * len(trainloader))
    #scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, epochs=epochs, steps_per_epoch=len(trainloader))
    criterion = nn.BCEWithLogitsLoss()
    
    best_loss = 1
    best_preds = []
    t = tqdm(range(epochs))
    for epoch in t:
        train_loss = train_model(model, optimizer, scheduler, criterion, trainloader)
        valid_loss, valid_preds = validate_model(model, criterion, validloader)
        t.set_postfix(train_loss=train_loss, valid_loss=valid_loss, lr=scheduler.get_last_lr()[0])
        
        if math.isnan(valid_loss):
            model.load_state_dict(torch.load("/best_model.pth"))
        if valid_loss < best_loss:
            best_loss = valid_loss
            best_preds = valid_preds
            torch.save(model.state_dict(), "/best_model.pth")
    
    model.load_state_dict(torch.load("/best_model.pth"))
    
    return model, best_loss, best_preds

In [29]:
epochs = 30
batch_size = 128

In [30]:
#https://www.kaggle.com/optimo/tabnetregressor-2-0-train-infer/data?
TABNET_MAX_EPOCH=200
tabnet_params = dict(n_d=32, n_a=32, n_steps=1, gamma=1.3,
                     lambda_sparse=0, optimizer_fn=LookaheadAdam, #torch.optim.Adam, #
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     cat_idxs=[0,1],cat_emb_dim=1,
                     mask_type='entmax',
                     scheduler_params=dict(mode="min",
                                           patience=5,
                                           min_lr=1e-5,
                                           factor=0.2,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=10,
                     momentum=0.1,
                     #n_independent=1,
                     #n_shared=1,
                     clip_value=1,
                     )

NameError: name 'LookaheadAdam' is not defined

In [ ]:
import math
kf = MultilabelStratifiedKFold(n_splits=7, shuffle=True, random_state=KFoldSeed)
models = []
all_submodels = []
losses = []
base_losses = []
masks = []
y_preds = []
tqdm.get_lock().locks = []
for train_index, valid_index in kf.split(X, y):
    X_train, y_train, y_train_nonscored = X[train_index], y[train_index], y_nonscored[train_index]
    X_valid, y_valid, y_valid_nonscored = X[valid_index], y[valid_index], y_nonscored[valid_index]
    
    train_dataset = MoADataset(X_train, y_train)
    train_nonscored_dataset = MoADataset(X_train, y_train_nonscored)
    valid_dataset = MoADataset(X_valid, y_valid)
    valid_nonscored_dataset = MoADataset(X_valid, y_valid_nonscored)
    
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    trainnonscoredloader = torch.utils.data.DataLoader(train_nonscored_dataset, batch_size=batch_size, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    validnonscoredloader = torch.utils.data.DataLoader(valid_nonscored_dataset, batch_size=batch_size, shuffle=False)
    
    
    
    model = MoAModel(
        num_features=len(X_train[0]),
        num_targets=len(y_train[0]),
        num_targets_nonscored=len(y_train_nonscored[0]),
        hidden_size=1024,
    )
    
    
    model, best_loss, best_preds = train_nn(model, trainloader, validloader, 30, 0.01)
    
    
    hiddenmodel = MoAModelHidden(
        num_features=len(X_train[0]),
        num_targets=len(y_train[0]),
        num_targets_nonscored=len(y_train_nonscored[0]),
        hidden_size=1024,
    )
    hiddenmodel.to(device)
    with torch.no_grad():
        hiddenmodel(torch.tensor(X_train).float().to(device))
    hiddenmodel, hidden_loss, hidden_preds = train_nn(hiddenmodel, trainloader, validloader, 30, 0.01)
    
    tabnetmodel = TabNetRegressor(**tabnet_params)
    tabnetmodel.fit(X_train=X_train,
              y_train=y_train,
              eval_set=[(X_valid, y_valid)],
              eval_name = ["val"],
              eval_metric = ["logits_ll"],
              max_epochs=TABNET_MAX_EPOCH,
              patience=20, batch_size=256, virtual_batch_size=256,
              num_workers=1, drop_last=False,
              # use binary cross entropy as this is not a regression problem
              loss_fn=torch.nn.functional.binary_cross_entropy_with_logits)
    
    tabnet_preds = sigmoid(tabnetmodel.predict(X_valid))
    
    weights = get_best_weights(y_valid, [best_preds, hidden_preds, tabnet_preds])
    
    weighted_loss = log_loss_numpy(y_valid, ensemble_with_weights(weights, [best_preds, hidden_preds, tabnet_preds]))
    losses.append(weighted_loss)
    print(weights)
    print(best_loss, log_loss_numpy(y_valid, best_preds), log_loss_numpy(y_valid, hidden_preds), log_loss_numpy(y_valid, tabnet_preds), weighted_loss)
    
    
    testdataset = MoATestDataset(X_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=batch_size, shuffle=False)
    
    
    pred1 = test_model(model, testloader)
    pred2 = test_model(hiddenmodel, testloader)
    pred3 = sigmoid(tabnetmodel.predict(X_test))
    y_preds.append(ensemble_with_weights(weights, [pred1, pred2, pred3]))

In [ ]:
print(losses)

In [ ]:
y_test = np.mean(y_preds, axis=0)

In [ ]:
df_sample_submission[:] = y_test
df_sample_submission[df_test_features['cp_type'] == 1] = 0
df_sample_submission

In [ ]:
#df_sample_submission.to_csv('submission.csv')